# Подготовка инструментов

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import random
import joblib
from sklearn.preprocessing import LabelEncoder
import datetime
from datetime import date
import numpy as np
import pandas as pd
import warnings
from fairness_pipeline import fairness_analysis, continuous_binning_calc

warnings.filterwarnings('ignore')


In [3]:
from copy import deepcopy

relu = np.vectorize(lambda x: x if x>0 else 0.)
sigm = lambda x: 1./(1+np.exp(-x))

vec_sizes = [45, 45, 45, 20, 15, 1]

def perceptron(xx, vec_sizes):
    
    xx_ext = xx
    generator = np.random.RandomState(43)
    
    for i, new_size in enumerate(vec_sizes):
        xx_ext = np.hstack([np.ones((xx_ext.shape[0], 1)), xx_ext])
        
        weights = generator.normal(0., 1, size=(xx_ext.shape[1], new_size))
        
        xx_ext = relu(xx_ext.dot(weights))
        
    return np.floor(xx_ext.reshape((-1,)))

# Генерация данных

In [4]:
# Генератор 
gen = np.random.RandomState(42)


# Генерация признаков
X = pd.DataFrame(data=gen.normal(50., 100., size=(30000, 30)),
                 columns=[f'feat_{i}' for i in range(30)],
                 index=[i for i in range(30000)])


# Имитация зависимости
y = pd.Series(data=1*(perceptron(X.values, [60, 32, 12, 5, 1])>0),
              index=[i for i in range(30000)],
              name='target'
             )

# Защищенные характеристики 
fair_factors = ['feat_3', 'feat_8', 'feat_13', 'feat_27']

# Остальные признаки
no_fair_factors = list(set(X.columns) - set(fair_factors))

Сгенеруем защищенные характеристики и "руками" внесем дискриминацию.
С помощью другого случайного перцептрона сымитируем зависимость 
одного защищенного фактора от признаков, не являющихся защищенными характеристиками

In [5]:
# Генерация
X['feat_3'] = gen.randint(18, 40, size=(30000,))

# Внесение "руками" дискриминации
X['feat_3'][y>0]=gen.randint(30, 41, size=(len(X['feat_3'][y>0]),))

# Генерация
X['feat_8'] = gen.choice([0, 1], size=(30000,), p=[0.4, 0.6])

# Внесение "руками" дискриминации
X['feat_8'][y>0]=gen.choice([0, 1], size=(len(X['feat_8'][y>0]),), p=[0.1, 0.9])

# Генерация
X['feat_13'] = gen.uniform(0, 10, size=(30000,))

# Имитация зависимости 
X['feat_27'] = perceptron(X[no_fair_factors].values, vec_sizes)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_oos, y_train, y_oos = train_test_split(X, y, 
                                                  test_size=0.1, 
                                                  random_state=42,
                                                  stratify=y)

### Fairness pipeline

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

Качество итоговой модели

In [8]:
-1 + 2*roc_auc_score(y_oos, model.predict_proba(X_oos)[:, 1])

0.8563304113977532

In [32]:
from myfairness import FairnessValidation

val = FairnessValidation(model,
                   train={'x': X_train, 'y': y_train},
                   oos={'x': X_oos, 'y': y_oos},
                   cat_features=list(X_train.columns[X_train.nunique() < 25]), 
                   protected_feats=fair_factors, n_bootstrap=64)
# val.test_list = [val.test_list[i] for i in [1, 2, 3]]

In [33]:
val.test_list

[<bound method FairnessValidation.test_indirect_discrimination of <myfairness.FairnessValidation object at 0x7f272e4bbe10>>,
 <bound method FairnessValidation.test_oppressed_privileged_ci of <myfairness.FairnessValidation object at 0x7f272e4bbe10>>,
 <bound method FairnessValidation.test_target_rate_delta of <myfairness.FairnessValidation object at 0x7f272e4bbe10>>,
 <bound method FairnessValidation.test_tprd_fprd_delta of <myfairness.FairnessValidation object at 0x7f272e4bbe10>>]

In [34]:
res = val.validate()
list(res.keys())

  0%|          | 0/4 [02:46<?, ?it/s]


NameError: name 'test_result' is not defined

In [12]:
train = pd.concat([X_train, y_train], axis=1)
train['score'] = model.predict_proba(X_train)[:, 1]

oos = pd.concat([X_oos, y_oos], axis=1)
oos['score'] = model.predict_proba(X_oos)[:, 1]

# Без разбиения на сегменты

In [13]:
from fairness_pipeline import fairness_analysis, continuous_binning_calc


result =  fairness_analysis(score = 'score',
                            target = 'target',
                            short_list = list(X_train.columns),
                            features_to_analyse = fair_factors, 
                            model_obj = model,
                            train = train, 
                            oos = oos, 
                            segm = None,  # в случае если модель разбивается по сегментам, сделать цикл по ним
                            logreg = None,  # задать не None, если это логрег
                            ) 

short_list ['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29']
features_to_analyse:  ['feat_3', 'feat_8', 'feat_13', 'feat_27']
Количество уникальных значений в фиче feat_3_gr =  12
Количество пропусков =  0.0
Количество уникальных значений в фиче feat_8_gr =  2
Количество пропусков =  0.0
Количество уникальных значений в фиче feat_13_gr =  20
Количество пропусков =  0.0
Количество уникальных значений в фиче feat_27_gr =  21
Количество пропусков =  0.0
Калибровка скора
Калибровка скора заняла: 0.0 минут
Применение библиотеки fairlearn
feat_3_gr Wed Dec  7 10:29:04 2022
feat_8_gr Wed Dec  7 10:29:06 2022
feat_13_gr Wed Dec  7 10:29:07 2022
feat_27_gr Wed Dec  7 10:29:09 2022
Применение библиотеки fairlearn заняло: 0.12 мин

100%|██████████| 100/100 [00:05<00:00, 17.28it/s]


Conf interval Gini
feat_8_gr
Вывчисление конф.интервала Gini заняло: 0.0 минут
dd FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'target',
       'score', 'feat_3_gr', 'feat_8_gr', 'feat_13_gr', 'feat_27_gr', 'bin',
       'score_calibr'],
      dtype='object')
FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_4', 'feat_5', 'feat_6', 'feat_7',
       'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_14', 'feat_15',
       'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21',
       'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_28',
       'feat_29'],
      dtype='object')


100%|██████████| 100/100 [00:01<00:00, 76.87it/s]


Conf interval Gini
feat_13_gr
Вывчисление конф.интервала Gini заняло: 0.0 минут
dd FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'target',
       'score', 'feat_3_gr', 'feat_8_gr', 'feat_13_gr', 'feat_27_gr', 'bin',
       'score_calibr'],
      dtype='object')
FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_4', 'feat_5', 'feat_6', 'feat_7',
       'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_14', 'feat_15',
       'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21',
       'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_28',
       'feat_29'],
      dtype='object')


100%|██████████| 100/100 [00:08<00:00, 11.87it/s]


Conf interval Gini
feat_27_gr
Вывчисление конф.интервала Gini заняло: 0.0 минут
dd FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'target',
       'score', 'feat_3_gr', 'feat_8_gr', 'feat_13_gr', 'feat_27_gr', 'bin',
       'score_calibr'],
      dtype='object')
FEATURES: Index(['feat_0', 'feat_1', 'feat_2', 'feat_4', 'feat_5', 'feat_6', 'feat_7',
       'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_14', 'feat_15',
       'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21',
       'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_28',
       'feat_29'],
      dtype='object')


100%|██████████| 100/100 [00:08<00:00, 11.36it/s]


Косвенная дискриминация заняла: 0.82 минут
Feature swap
cat_feats []
Gini initial:  0.8563304113977532
Swap и новый предикт заняли: 0.01 минут

shape_sw (152, 4)
shape_old (152, 39)
Вспомогательные вычисления заняли 0.0 минут
Swap и новый предикт заняли: 0.01 минут

shape_sw (415, 4)
shape_old (415, 40)
Вспомогательные вычисления заняли 0.0 минут
Swap и новый предикт заняли: 0.01 минут

shape_sw (150, 4)
shape_old (150, 40)
Вспомогательные вычисления заняли 0.0 минут
Swap и новый предикт заняли: 0.01 минут

shape_sw (32, 4)
shape_old (32, 40)
Вспомогательные вычисления заняли 0.0 минут


# Результаты

In [14]:
from result_aggs import result_aggr

test_1, test_2, test_3, test_4 = result_aggr(result)

In [15]:
pass

In [16]:
test_1

,Защищенная характеристика,Правая граница CI_95 Gini,Защищенная характеристика не проверяется на дискриминацию,Защищенная характеристика проверяется на дискриминацию
0,feat_3_gr,0.258489,NaN,no
1,feat_8_gr,0.29311,NaN,no
2,feat_13_gr,0.24749,NaN,no
3,feat_27_gr,0.737881,NaN,yes


In [17]:
res['test_target_rate_delta']

,Защищенная характеристика,Минимальное относительное изменение прогнозного значения,Максимальное относительное изменение прогнозного значения,Результат теста
0,feat_3,-0.022222,0.017778,green
1,feat_8,-0.549296,1.330579,yellow
2,feat_13,0.000000,0.000000,green
3,feat_27,-0.079646,0.071429,green


In [18]:
test_2

,Защищенная характеристика,Минимальное относительное изменение прогнозного значения,Максимальное относительное изменение прогнозного значения,Результат теста
0,feat_3_gr,-0.999999,848684,red
1,feat_8_gr,-0.0696638,0.151203,green
2,feat_13_gr,0,0,green
3,feat_27_gr,0,0,green


In [19]:
res['test_tprd_fprd_delta']

,Защищенная характеристика,Изменение TPRD после перестановки значений,Изменение FPRD после перестановки значений,Результат теста
0,feat_3,0.005857,0.000000,green
1,feat_8,0.044392,0.014562,green
2,feat_13,0.000000,0.100000,yellow
3,feat_27,0.130994,0.000000,yellow


In [20]:
test_3

,Защищенная характеристика,Изменение TPRD после перестановки значений,Изменение FPRD после перестановки значений,Результат теста
0,feat_3_gr,-1,0.848684,grey
1,feat_8_gr,0.0180651,-0.054736,grey
2,feat_13_gr,0,0,grey
3,feat_27_gr,0,0,grey


In [21]:
res['test_oppressed_privileged_ci']

,Защищенная характеристика,95.0%-ый доверительный интервал для угнетаемой группы,95.0%-ый доверительный интервал для привелегированной группы,99.0%-ый доверительный интервал для угнетаемой группы,99.0%-ый доверительный интервал для привелегированной группы,Результат теста
0,feat_3,"(0.5435168731860663, 0.8030770363369052)","невозможно расчитать, в группе один класс","(0.5027370160198662, 0.8438568935031053)","невозможно расчитать, в группе один класс",gray
1,feat_8,"(0.8567995728101081, 0.9374679752546846)","(0.7758958093784335, 0.8338251870515293)","(0.844125648238703, 0.9501418998260898)","(0.7667944445212121, 0.8429265519087507)",green
2,feat_13,"(0.8043289210620156, 0.9503330835999891)","(0.4055082663681193, 0.8417644609046078)","(0.7813900046208623, 0.9732720000411424)","(0.33696745068551637, 0.9103052765872107)",green
3,feat_27,"(0.8454680522363367, 0.9679647835845586)","(0.6408432901328749, 0.9122071077451093)","(0.8262224209072953, 0.9872104149136001)","(0.5982089444460493, 0.9548414534319348)",green


In [22]:
test_4

,Защищенная характеристика,95%-ый доверительный интервал для угнетаемой группы,99%-ый доверительный интервал для угнетаемой группы,95%-ый доверительный интервал для привелегированной группы,99%-ый доверительный интервал для привелегированной группы,Результат теста
0,feat_3_gr,невозможно рассчитать_0.0,невозможно рассчитать_0.0,невозможно рассчитать_1.0,невозможно рассчитать_1.0,one_ex
1,feat_8_gr,"[0.8567988316460765, 0.9374687164187162]","[0.8441221354675189, 0.9501454125972738]","[0.7758952771356991, 0.8338257192942637]","[0.7667919219393532, 0.8429290744906096]",green
2,feat_13_gr,"[0.8245544461706559, 0.9614257027127185]","[0.8030461058569032, 0.9829340430264712]","[0.5494168009222644, 0.8688875224915478]","[0.49921425896137706, 0.9190900644524351]",green
3,feat_27_gr,"[0.6112646629519518, 1.0297609780736896]","[0.5455009562899644, 1.095524684735677]","[1.0, 1.0]","[1.0, 1.0]",green


## В случае разбиения на сегменты:

In [23]:
# колонка с сегментом должна быть названа как 'segment'! 

for seg in ['VOICE_CASH', 'ROBOT_CASH', 'WAIT_CARD', 'WAIT_CASH', 'CALL_CARD', 'VOICE_CARD', 'SMS_CASH', 'SMS_CARD',
         'ROBOT_CARD', 'WAIT_MORT', 'VOICE_MORT', 'SMS_MORT', 'EMAIL_CARD', 'CALL_CASH', 'CALL_MORT', 'EMAIL_CASH',
         'EMAIL_MORT', 'LETTER_MORT_CASH_CARD']:
    
    result = fairness_analysis(score = score_name,
                            target = target_name,
                            short_list = short_list,
                            features_to_analyse = fair_features, 
                            model_obj = model_obj,
                            train = train, 
                            oos = oot_or_oos, 
                            segm = seg,  # в случае если модель разбивается по сегментам, сделать цикл по ним
                            logreg = None,  # задать не None, если это логрег
                            ) 
    result['N_model'] = f'{model_number}_{seg}'
    result['date'] = date.today()
    result.to_csv(f'result_fair_{model_number}_{seg}.csv', index = False)

NameError: name 'score_name' is not defined

### Записать значение uplift для каждой фичи

In [ ]:
res_uplift = {}
result['Uplift'] = '-'
for k in res_uplift.keys():
    result.loc[result[''] == k, 'Uplift'] = res_uplift[k]

# В случае отсутствия дискриминационных фичей в модели: 

In [ ]:
result = no_discrimination_save(model_number)
result.to_csv(f'result_fair_{model_number}_{seg}.csv', index = False)